In [1]:
import csv
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM, Dropout,Embedding, Bidirectional
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('arabic'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
vocab_size = 5000
embedding_dim = 128
max_length = 200
oov_tok = '<OOV>' #  Out of Vocabulary
training_portion = 0.8

In [3]:
articles = []
labels = []

with open("Data/arabic_main.csv", 'r',encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        try:
            labels.append(row[1])   
        except:
            continue
        article = row[0]
        for word in STOPWORDS:
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)

In [4]:
len(labels),len(articles)

(8276, 8276)

In [5]:
print(labels[0])

weather


In [6]:
train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

In [7]:
print('train_articles' ,len(train_articles))
print('train_labels', len(train_labels))
print('validation_articles', len(validation_articles))
print('validation_labels', len(validation_labels))

train_articles 6620
train_labels 6620
validation_articles 1656
validation_labels 1656


In [8]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index

In [9]:
import pickle
filename = f'../../utils/tokenizer.sav'
pickle.dump(tokenizer, open(filename, 'wb'))

In [10]:
word_index

{'<OOV>': 1,
 'هل': 2,
 'ما': 3,
 'الطقس': 4,
 'الساعة': 5,
 '،': 6,
 'سيكون': 7,
 'الخاص': 8,
 'يوم': 9,
 'يمكنك': 10,
 'التقويم': 11,
 'اليوم': 12,
 'موعد': 13,
 'مساءً': 14,
 'الأسبوع': 15,
 'كيف': 16,
 'التحقق': 17,
 'من': 18,
 'فضلك': 19,
 'وقت': 20,
 'أريد': 21,
 'ستكون': 22,
 'توقعات': 23,
 'الحدث': 24,
 'بي': 25,
 '30': 26,
 'الجو': 27,
 'ماذا': 28,
 'إضافة': 29,
 'حدد': 30,
 'موعدًا': 31,
 'قل': 32,
 'لدي': 33,
 'طبيب': 34,
 'أود': 35,
 'مارس': 36,
 'أعرف': 37,
 'يمكن': 38,
 'أحتاج': 39,
 'الأحداث': 40,
 '3': 41,
 '4': 42,
 'مارس؟': 43,
 'تاريخ': 44,
 'صباحًا': 45,
 'متى': 46,
 'تخبرني': 47,
 'الظهر': 48,
 'يرجى': 49,
 'اليوم؟': 50,
 'الشهر': 51,
 'نعم': 52,
 'القادم': 53,
 'المقبل': 54,
 'درجة': 55,
 'شكرا': 56,
 '5': 57,
 'ليوم': 58,
 'أخبرني': 59,
 '11': 60,
 'الحياة': 61,
 'أكون': 62,
 'في': 63,
 'الثاني': 64,
 'تحقق': 65,
 'كم': 66,
 'عندما': 67,
 '12': 68,
 'الوقت': 69,
 'منبه': 70,
 'أنا': 71,
 'يمكنني': 72,
 'يكون': 73,
 'يجب': 74,
 'لديك': 75,
 'شكرًا': 76,
 '15': 77,

In [11]:
tokenizer.texts_to_sequences(['what will be the weather tommorrow morning ?'])

[[1, 1, 1, 1162, 1, 1, 1]]

In [12]:
train_sequences = tokenizer.texts_to_sequences(train_articles)
len(train_sequences)

6620

In [13]:
train_sequences[0]

[3, 4, 252, 54]

In [14]:
train_padded = pad_sequences(train_sequences, maxlen=max_length,)

In [15]:
len(train_padded),len(train_padded[0])

(6620, 200)

In [16]:
validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length)

In [17]:
set(labels)

{'general', 'greeting', 'recommendation', 'schedule', 'thank', 'weather'}

In [18]:
label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))

In [19]:
validation_label_seq

array([[3],
       [1],
       [3],
       ...,
       [2],
       [1],
       [2]])

In [20]:
# train_labels[744], training_label_seq[744]

In [21]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim ))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(embedding_dim)))
model.add(Dense(7,activation='softmax' ))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 128)         640000    
                                                                 
 dropout (Dropout)           (None, None, 128)         0         
                                                                 
 bidirectional (Bidirectiona  (None, 256)              263168    
 l)                                                              
                                                                 
 dense (Dense)               (None, 7)                 1799      
                                                                 
Total params: 904,967
Trainable params: 904,967
Non-trainable params: 0
_________________________________________________________________


In [22]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

C:\Users\dell\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [23]:
num_epochs = 10
history = model.fit(train_padded, training_label_seq, epochs=num_epochs, validation_data=(validation_padded, validation_label_seq), verbose=2)

Epoch 1/10
207/207 - 84s - loss: 0.7252 - accuracy: 0.7356 - val_loss: 0.3080 - val_accuracy: 0.9161 - 84s/epoch - 406ms/step
Epoch 2/10
207/207 - 81s - loss: 0.1970 - accuracy: 0.9440 - val_loss: 0.1930 - val_accuracy: 0.9505 - 81s/epoch - 389ms/step
Epoch 3/10
207/207 - 78s - loss: 0.0841 - accuracy: 0.9751 - val_loss: 0.1672 - val_accuracy: 0.9523 - 78s/epoch - 375ms/step
Epoch 4/10
207/207 - 73s - loss: 0.0527 - accuracy: 0.9834 - val_loss: 0.1542 - val_accuracy: 0.9626 - 73s/epoch - 354ms/step
Epoch 5/10
207/207 - 88s - loss: 0.0341 - accuracy: 0.9906 - val_loss: 0.1801 - val_accuracy: 0.9523 - 88s/epoch - 424ms/step
Epoch 6/10
207/207 - 91s - loss: 0.0256 - accuracy: 0.9908 - val_loss: 0.1997 - val_accuracy: 0.9505 - 91s/epoch - 441ms/step
Epoch 7/10
207/207 - 89s - loss: 0.0196 - accuracy: 0.9934 - val_loss: 0.2131 - val_accuracy: 0.9505 - 89s/epoch - 430ms/step
Epoch 8/10
207/207 - 88s - loss: 0.0168 - accuracy: 0.9950 - val_loss: 0.2253 - val_accuracy: 0.9426 - 88s/epoch - 424

In [24]:
txt_list = ["ما حال الطقس اليوم ؟","احجز لى معاد اليوم الساعة 5","كان بوم جميل جدا"," اقترح فيلم جديد لاشاهده"]
labels = ['schedule','general','weather','recommendation','greeting','thank']
answer = list()
for txt in txt_list:
    seq = tokenizer.texts_to_sequences([txt])
    padded = pad_sequences(seq, maxlen=max_length)
    pred = model.predict(padded)
    try:
        label = labels[np.argmax(pred)-1] 
        answer.append(label)
    except:
        answer.append('general')
print(answer)

1/1 [==============================] - 0s 39ms/step
['weather', 'schedule', 'general', 'recommendation']


In [25]:
txt_list = ["انا حزين جدا","سوف ارسم قطتى حتى انسى","ابقى فكرنى بمعاد الدكتور بكرة","الجو عامل ايه فى المانيا","عايز فيلم اكشن جديد اتفرج عليه"]

answer = list()
for txt in txt_list:
    seq = tokenizer.texts_to_sequences([txt])
    padded = pad_sequences(seq, maxlen=max_length)
    pred = model.predict(padded)
    try:
        label = labels[np.argmax(pred)-1] 
        answer.append(label)
    except:
        answer.append('general')
print(answer)

1/1 [==============================] - 0s 38ms/step
['general', 'general', 'schedule', 'weather', 'recommendation']


In [29]:
model.save("models")

INFO:tensorflow:Assets written to: models\assets


INFO:tensorflow:Assets written to: models\assets


In [30]:
from tensorflow import keras

m = keras.models.load_model("models")

In [31]:
answer = list()
for txt in txt_list:
    seq = tokenizer.texts_to_sequences([txt])
    padded = pad_sequences(seq, maxlen=max_length)
    pred = m.predict(padded)
    try:
        label = labels[np.argmax(pred)-1] 
        answer.append(label)
    except:
        answer.append('general')
print(answer)

1/1 [==============================] - 0s 37ms/step
['general', 'general', 'schedule', 'weather', 'recommendation']
